本章就要深入学习pandas的高级功能。

# 12.1 分类数据 Categorical Data----------------------------------------------------------------

# 背景和目的

表中的一列通常会有重复的包含不同值的小集合的情况。我们已经学过了unique和value_counts，它们可以从数组提取出不同的值，并分别计算频率：

许多数据系统（数据仓库、统计计算或其它应用）都发展出了特定的表征重复值的方法，以进行高效的存储和计算。在数据仓库中，最好的方法是使用所谓的包含不同值的维表(Dimension Table)，将主要的参数存储为引用维表整数键：

可以使用**take方法**存储原始的字符串Series：

这种用整数表示的方法称为分类或字典编码表示法。不同值的数组称为分类、字典或数据级。本书中，我们使用分类的说法。表示分类的整数值称为分类编码或简单地称为编码。

In [1]:
import pandas as pd
import numpy as np

In [2]:
values = pd.Series([0, 1, 0, 0] * 2)
dim = pd.Series(['apple', 'orange'])
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

# pandas的分类类型

pandas有一个特殊的分类类型，用于保存使用整数分类表示法的数据。看一个之前的Series例子：

这里，df['fruit']是一个Python字符串对象的数组。我们可以通过调用它，将它转变为分类：

【**astype方法**改变数据的dtype】

fruit_cat的值不是NumPy数组，而是一个pandas.Categorical实例：

Categorical实例分类对象有**categories属性**和**codes属性**：

In [3]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
N = len(fruits)
df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': np.random.randint(3, 15, size=N),
                   'weight': np.random.uniform(0, 4, size=N)},
                  columns=['basket_id', 'fruit', 'count', 'weight'])
df['fruit']

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: object

In [4]:
fruit_cat  = df['fruit'].astype('category')
fruit_cat 

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [5]:
df['fruit'].values

array(['apple', 'orange', 'apple', 'apple', 'apple', 'orange', 'apple',
       'apple'], dtype=object)

In [6]:
fruit_cat.values

[apple, orange, apple, apple, apple, orange, apple, apple]
Categories (2, object): [apple, orange]

In [7]:
fruit_cat.values.categories

Index(['apple', 'orange'], dtype='object')

In [8]:
fruit_cat.values.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

你可将DataFrame的列通过分配转换结果，转换为分类：

你还可以从其它Python序列**直接创建pandas.Categorical**：

如果你已经从其它源获得了分类编码，你还可以使用**from_codes构造器**：

与显示指定不同，分类变换不认定指定的分类顺序。因此取决于输入数据的顺序，categories数组的顺序会不同。当使用from_codes或其它的构造器时，你可以指定分类一个有意义的顺序：【使用**参数ordered**=True】

输出[foo < bar < baz]指明‘foo’位于‘bar’的前面，以此类推。无序的分类实例可以通过**as_ordered方法**排序：

最后要注意，分类数据不必须是字符串，尽管我仅仅展示了字符串的例子。分类数组可以包括任意不可变类型。

In [9]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_categories

[foo, bar, baz, foo, bar]
Categories (3, object): [bar, baz, foo]

In [10]:
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]
my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

In [11]:
ordered_cat = pd.Categorical.from_codes(codes, categories, ordered=True)
ordered_cat

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

In [12]:
my_cats_2.as_ordered()

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

# 用分类进行计算

与非编码版本（比如字符串数组）相比，使用pandas的Categorical有些类似。某些pandas组件，比如groupby函数，更适合进行分类。还有一些函数可以使用有序标志位。

来看一些随机的数值数据，使用pandas.qcut面元函数。它会返回pandas.Categorical，我们之前使用过pandas.cut，但没解释分类是如何工作的：

计算这个数据的分位面元，提取一些统计信息：

虽然有用，确切的样本分位数与分位的名称相比，不利于生成汇总。我们可以使用labels参数qcut，实现目的：【使用**参数labels**，将样本分位数用标签名称表示】

加上标签的面元分类不包含数据面元边界的信息，因此可以使用groupby提取一些汇总信息：【使用参数name，指定其名称】

分位数列保存了原始的面元分类信息，包括排序：

In [13]:
np.random.seed(12345)
draws = np.random.randn(1000)
bins = pd.qcut(draws, 4)
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.95, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.95, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

In [14]:
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
bins

[Q2, Q3, Q2, Q2, Q4, ..., Q3, Q2, Q1, Q3, Q4]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [15]:
bins = pd.Series(bins, name='quartile')
results = pd.Series(draws).groupby(bins).agg(['count', 'min', 'max']).reset_index()
results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


In [16]:
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

# 用分类提高性能

如果你是在一个特定数据集上做大量分析，*将其转换为分类可以极大地提高效率*。DataFrame列的分类使用的内存通常少的多。来看一些包含一千万元素的Series，和一些不同的分类：

现在，将标签转换为分类：这时，可以看到标签使用的内存远比分类多：

转换为分类不是没有代价的，但这是一次性的代价：【astype方法时消耗时间】

**GroupBy使用分类操作明显更快**，是因为底层的算法使用整数编码数组，而不是字符串数组。

In [17]:
N = 10000000
draws = pd.Series(np.random.randn(N))
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))
categories = labels.astype('category')
labels.memory_usage()

40000042

In [18]:
categories.memory_usage()

10000154

In [19]:
%time _ = labels.astype('category')

Wall time: 904 ms


# 分类方法 Categorical Methods

包含分类数据的Series有一些特殊的方法，类似于Series.str字符串方法。它还提供了方便的分类和编码的使用方法。看下面的Series：

特别的**cat属性**提供了分类方法的入口：【也能通过values属性也能获取categories属性和codes属性】

假设我们知道这个数据的实际分类集，超出了数据中的四个值。我们可以使用**set_categories方法**改变它们：【没对应好的话，会产生缺失，或者新的分类】

虽然数据看起来没变，新的分类将反映在它们的操作中。例如，如果有的话，value_counts表示分类：

在大数据集中，分类经常作为节省内存和高性能的便捷工具。过滤完大DataFrame或Series之后，许多分类可能不会出现在数据中。我们可以使用remove_unused_categories方法删除没看到的分类：

> 表12-1列出了可用的分类方法。

> add_categories | as_ordered | as_unordered | remove_categories | remove_unused_categories |

> rename_categories | set_categories |

In [20]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

In [21]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [22]:
cat_s.values.codes

array([0, 1, 2, 3, 0, 1, 2, 3], dtype=int8)

In [23]:
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]

In [24]:
cat_s.value_counts()

d    2
c    2
b    2
a    2
dtype: int64

In [25]:
cat_s2.value_counts()

d    2
c    2
b    2
a    2
e    0
dtype: int64

In [26]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])] # 作个分类过滤
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): [a, b, c, d]

In [27]:
cat_s3.cat.remove_unused_categories()
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): [a, b, c, d]

### 为建模创建虚拟变量

当你使用统计或机器学习工具时，通常会将分类数据转换为虚拟变量，也称为one-hot编码。这包括创建一个不同类别的列的DataFrame；这些列包含给定分类的1s，其它为0。

前面的第7章提到过，**pandas.get_dummies函数**可以转换这个以为分类数据为包含虚拟变量的DataFrame：

In [28]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

In [29]:
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


# 12.2 GroupBy高级应用 Advanced GroupBy Use-----------------------------------------

# 分组转换和解封分组 Group Transforms and “Unwrapped” GroupBys

在第10章，我们在分组操作中学习了apply方法，进行转换。还有另一个**transform方法**，它与apply很像，但是对使用的函数有一定限制：

1. 它可以产生**向分组形状广播**标量值
2. 它可以产生一个和输入组形状相同的对象
3. 它不能修改输入

来看一个简单的例子：按键进行分组：

假设我们想产生一个和df['value']形状相同的Series，但值替换为按键分组的平均值。我们可以传递函数lambda x: x.mean()进行转换：

对于内置的聚合函数，我们可以传递一个字符串假名作为GroupBy的agg方法：

与apply类似，transform的函数会返回Series，但是结果必须与输入大小相同。举个例子，我们可以用lambda函数将每个分组乘以2：

再举一个复杂的例子，我们可以计算每个分组的降序排名：

In [30]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12.)})

g = df.groupby('key')['value']
dict(list(g))    # g.size()

{'a': 0    0.0
 3    3.0
 6    6.0
 9    9.0
 Name: value, dtype: float64, 'b': 1      1.0
 4      4.0
 7      7.0
 10    10.0
 Name: value, dtype: float64, 'c': 2      2.0
 5      5.0
 8      8.0
 11    11.0
 Name: value, dtype: float64}

In [31]:
g.transform(lambda x: x.mean())

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [32]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [33]:
g.transform(lambda x: x * 2)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [34]:
g.transform(lambda x : x.rank(ascending=False))

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

看一个由简单聚合构造的的分组转换函数：

我们用transform或apply可以获得等价的结果：

内置的聚合函数，比如mean或sum，通常比apply函数快，也比transform快。这允许我们进行一个所谓的解封（unwrapped）分组操作：

解封分组操作可能包括多个分组聚合，但是**矢量化操作**还是会带来收益。

In [35]:
def normalize(x):
    return (x - x.mean()) / x.std()

In [36]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [37]:
g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [38]:
normalized = (df['value']-g.transform('mean')) / g.transform('std')
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

# 分组的时间重采样

对于时间序列数据，**resample方法**从语义上是一个基于内在时间的分组操作。下面是一个示例表：

这里，我们可以用time作为索引，然后重采样：

假设DataFrame包含多个时间序列，用一个额外的分组键的列进行标记：

要对每个key值进行相同的重采样，我们引入**pandas.TimeGrouper对象**：

我们然后设定时间索引，用key和time_key分组，然后聚合：

使用TimeGrouper的限制是：时间必须是Series或DataFrame的索引。

In [39]:
N = 15
times = pd.date_range('2017-05-20 00:00', freq='1min', periods=N)
df = pd.DataFrame({'time': times, 'value': np.arange(N)})
df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


In [40]:
df.set_index('time').resample('5min').count()

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


In [41]:
df2 = pd.DataFrame({'time': times.repeat(3),
                    'key': np.tile(['a', 'b', 'c'], N),
                    'value': np.arange(N * 3.)})
df2

,key,time,value
0,a,2017-05-20 00:00:00,0.0
1,b,2017-05-20 00:00:00,1.0
2,c,2017-05-20 00:00:00,2.0
3,a,2017-05-20 00:01:00,3.0
4,b,2017-05-20 00:01:00,4.0
5,c,2017-05-20 00:01:00,5.0
6,a,2017-05-20 00:02:00,6.0
7,b,2017-05-20 00:02:00,7.0
8,c,2017-05-20 00:02:00,8.0
9,a,2017-05-20 00:03:00,9.0


In [42]:
time_key = pd.TimeGrouper('5min')
resampled = df2.set_index('time').groupby(['key', time_key]).sum()
# dict(list(resampled))
resampled

value
key time                      
a   2017-05-20 00:00:00   30.0
    2017-05-20 00:05:00  105.0
    2017-05-20 00:10:00  180.0
b   2017-05-20 00:00:00   35.0
    2017-05-20 00:05:00  110.0
    2017-05-20 00:10:00  185.0
c   2017-05-20 00:00:00   40.0
    2017-05-20 00:05:00  115.0
    2017-05-20 00:10:00  190.0

# 12.3 链式编程技术 Techniques for Method Chaining-------------------------------------

当对数据集进行一系列变换时，你可能发现创建的多个临时变量其实并没有在分析中用到。

**DataFrame.assign方法**是一个df[k] = v形式的函数式的列分配方法。它不是就地修改对象，而是返回新的修改过的DataFrame。因此，下面的语句是等价的：
```
# Usual non-functional way
df2 = df.copy()
df2['k'] = v

# Functional assign way
df2 = df.assign(k=v)
```

就地分配可能会比assign快，但是assign可以方便地进行链式编程：
```
result = (df2.assign(col1_demeaned=df2.col1 - df2.col2.mean())
          .groupby('key')
          .col1_demeaned.std())

```
对应原来的写法：
```python
df = load_data()
df2 = df[df['col2'] < 0]
df2['col1_demeaned'] = df2['col1'] - df2['col1'].mean()
result = df2.groupby('key').col1_demeaned.std()
```

assign和许多其它pandas函数可以接收类似函数的参数，即可调用对象（callable）。

原来的写法
```python
df = load_data()
df2 = df[df['col2'] < 0]
```

可以重写为：
```
df = (load_data()
      [lambda x: x['col2'] < 0])
```
这里，load_data的结果没有赋值给某个变量，因此传递到[ ]的函数在这一步被绑定到了对象。

我们可以把整个过程写为一个单链表达式：
```python
result = (load_data()
          [lambda x: x.col2 < 0]
          .assign(col1_demeaned=lambda x: x.col1 - x.col1.mean())
          .groupby('key')
          .col1_demeaned.std())
```


# 管道方法 The pipe Method

你可以用Python内置的pandas函数和方法，用带有可调用对象的链式编程做许多工作。但是，有时你需要使用自己的函数，或是第三方库的函数。这时就要用到管道方法。

当使用接收、返回Series或DataFrame对象的函数式，你可以调用**Dataframe.pipe方法**将其重写：

f(df)和df.pipe(f)是等价的，但是pipe使得链式声明更容易。

原来的写法：
```python
a = f(df, arg1=v1)
b = g(a, v2, arg3=v3)
c = h(b, arg4=v4)
```

重写后
```python
result = (df.pipe(f, arg1=v1)
          .pipe(g, v2, arg3=v3)
          .pipe(h, arg4=v4))
```

pipe的另一个有用的地方是提炼操作为可复用的函数。看一个从列减去分组方法的例子：
```python
g = df.groupby(['key1', 'key2'])
df['col1'] = df['col1'] - g.transform('mean')
```

假设你想转换多列，并修改分组的键。另外，你想用链式编程做这个转换。下面就是一个方法：
```python
def group_demean(df, by, cols):
    result = df.copy()
    g = df.groupby(by)
    for c in cols:
        result[c] = df[c] - g[c].transform('mean')
    return result
```

重写为：
```python
result = (df[df.col1 < 0]
          .pipe(group_demean, ['key1', 'key2'], ['col1']))
```